In [13]:
import json
import pandas as pd

In [14]:
dataset = None
with open("sample.json", "r") as read_content: 
    dataset = (json.load(read_content))

In [15]:
print(dataset.keys())

dict_keys(['FL', 'DV'])


In [16]:
data = {f"FL_{case}":dataset["FL"][case] for case in dataset["FL"]}
data.update({f"DV_{case}":dataset["DV"][case] for case in dataset["DV"]})
data.keys()

dict_keys(['FL_2', 'FL_3', 'FL_4', 'FL_6', 'FL_8', 'FL_9', 'FL_14', 'FL_15', 'FL_16', 'FL_19', 'FL_26', 'FL_28', 'FL_30', 'FL_33', 'FL_35', 'FL_38', 'FL_43', 'FL_45', 'FL_47', 'FL_51', 'FL_52', 'FL_53', 'FL_54', 'FL_55', 'FL_56', 'FL_57', 'FL_58', 'FL_59', 'FL_60', 'FL_61', 'FL_62', 'FL_63', 'FL_64', 'FL_65', 'FL_66', 'FL_67', 'FL_68', 'FL_69', 'FL_70', 'FL_71', 'FL_72', 'FL_73', 'FL_74', 'FL_75', 'FL_76', 'FL_77', 'FL_78', 'FL_79', 'FL_80', 'FL_81', 'FL_82', 'FL_83', 'FL_84', 'FL_85', 'FL_86', 'FL_87', 'FL_88', 'FL_89', 'FL_90', 'FL_91', 'FL_92', 'FL_93', 'FL_94', 'FL_95', 'FL_96', 'FL_97', 'FL_98', 'FL_99', 'FL_100', 'FL_101', 'FL_102', 'FL_103', 'FL_104', 'FL_105', 'FL_106', 'FL_107', 'FL_108', 'FL_109', 'FL_110', 'FL_111', 'FL_112', 'FL_113', 'FL_114', 'FL_115', 'FL_116', 'FL_117', 'FL_118', 'FL_119', 'FL_120', 'FL_121', 'FL_122', 'FL_123', 'FL_124', 'FL_125', 'FL_126', 'FL_127', 'FL_128', 'FL_129', 'FL_130', 'FL_131', 'FL_132', 'FL_133', 'FL_135', 'FL_136', 'FL_137', 'FL_138', 'FL

In [17]:
processed = []

def make_entry(case_id, petitioner_rep, respondent_rep, restraining_order_terms, relevant_comments):
    return {
        "case_id": case_id,
        "petitioner_rep": petitioner_rep, 
        "respondent_rep": respondent_rep,
        "restraining_order_terms": restraining_order_terms,
        "relevant_comments": relevant_comments
        }

restraining_order_terms = ["RESTRAINING","ORDER","ROAH","DV-130"]

def count_occurences(haystack, needle):
    try:
        return haystack.index(needle)
    except:
        return 0
    
for idx,case in enumerate(data):
    case_object = data[case]
    case_id = case_object["event"]["data"]["caseNumber"]
    petitioners = []
    respondents = []
    for party in case_object["event"]["data"]["caseParties"]:
        if party["type"] == "Petitioner":
            petitioners.append(party)
        if party["type"] == "Respondent":
            respondents.append(party)
    petitioners.sort(key=lambda x: len(x["fullName"]), reverse=True)
    respondents.sort(key=lambda x: len(x["fullName"]), reverse=True)
    if not (len(petitioners) == 0 or len(respondents) == 0):
        petitioner_full_name = petitioners[0]["fullName"]
        respondent_full_name = respondents[0]["fullName"]
        attornies = {}
        for attorney in case_object["event"]["data"]['caseAttornies']:
            representing = attorney["representing"]
            attornies[representing] = attorney
    ro_occured = False
    relevant_comments = []
    for event in case_object["event"]["data"]["caseEvents"]:
        comm = event["comment"] if event["comment"] is not None else ""
        comm = comm.upper()
        event_type = event["type"] if event["type"] is not None else ""       
        event_type = event_type.upper()
        occurences = [count_occurences(comm, term.upper()) for term in restraining_order_terms]
        occurences = occurences + [count_occurences(event_type, term.upper()) for term in restraining_order_terms]
        if not ro_occured and sum(occurences) != 0:
            ro_occured = True
        if sum(occurences) != 0:
            relevant_comments.append(event_type)
            relevant_comments.append(comm)
            relevant_comments.append("\n")
    respondent_attorney = respondent_full_name in attornies
    petitioner_attorney = petitioner_full_name in attornies

    #Post Processing
    relevant_comments = "\n".join(relevant_comments)


    processed.append(make_entry(case_id,petitioner_attorney,respondent_attorney,ro_occured,relevant_comments))
dataframe = pd.DataFrame(processed)
dataframe

,case_id,petitioner_rep,respondent_rep,restraining_order_terms,relevant_comments
0,23FL000002,True,True,False,
1,23FL000003,True,True,False,
2,23FL000004,False,False,False,
3,23FL000006,False,False,False,
4,23FL000008,False,False,False,
...,...,...,...,...,...
4871,23DV001064,False,True,False,
4872,23DV001065,False,False,False,
4873,23DV001066,False,False,False,
4874,23DV001067,False,False,False,


In [18]:
dataframe.to_csv("processed.csv")